In [1]:
args = {
    "dataset_folder": "./train_10M",
   
}

In [2]:
import os
from datasets import load_dataset
import datasets
import torch
import json

In [3]:
!mkdir curricula

mkdir: cannot create directory ‘curricula’: File exists


### Curriculum 2023 dataset

The exact data from Thoma et al 2023. Note that the model does not see all examples in each epoch.

In [4]:
from datasets import load_dataset
import os
os.environ['HF_HOME'] = '/data/loriss21dm/cache'
# dev = load_dataset("babylm-anon/dev-data")

In [5]:
LAST_GEN_DIR = "last_gen"
EPOCHS_PER_STAGE = 10
datasets_stages = []
orders_stages = []

torch.manual_seed(0)
for folder in (sorted(os.listdir(LAST_GEN_DIR))):
    d = load_dataset("text", data_dir =os.path.join(LAST_GEN_DIR, folder), cache_dir='/data/loriss21dm/cache')["train"] 
    d = d.shuffle() # we shuffle within the stage
    d = datasets.Dataset.from_dict(d[0:len(d)//10]) # TODO
    datasets_stages.append(d)
    offset = orders_stages[-1][-1]+1 if len(orders_stages) else 0
    for _ in range(0,EPOCHS_PER_STAGE):
        orders_stages.append(torch.arange(offset,len(d)+offset))
    


dataset = datasets.concatenate_datasets(datasets_stages)

dataset.save_to_disk("./curricula/datasets/curriculum_100M_2023")
#assert torch.equal(torch.cat([o.flatten() for o in orders_stages]),torch.arange(0,len(dataset)))

torch.save(orders_stages, "./curricula/curriculum_100M_2023")
len(orders_stages)

Saving the dataset (0/1 shards):   0%|          | 0/1058753 [00:00<?, ? examples/s]

40

In [6]:
dataset

Dataset({
    features: ['text'],
    num_rows: 1058753
})

In [7]:
sum((len(o) for o in orders_stages)) / 10587551 


0.99999801653848

In [8]:
dataset

Dataset({
    features: ['text'],
    num_rows: 1058753
})

In [9]:
torch.tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35]).repeat(1,10,1),

(tensor([[[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
           17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
           34, 35],
          [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
           17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
           34, 35],
          [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
           17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
           34, 35],
          [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
           17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
           34, 35],
          [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
           17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
           34, 35],
          [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
           17, 18, 19, 20, 21, 

### Curriculum with 10M 2024 dataset

For training with the strategy in Thoma et al 2023. Note that the model does not see all examples in each epoch.

This is not designed to extract influence estimates from

In [10]:
args["raw_dataset_folder_babylm"] = "./train_10M"
args["dataset_folder"] = "./curricula/datasets/curriculum_10M_2024"
args["curriculum_path"] = "./curricula/curriculum_10M_2024"
args["eval_dataset_folder"] = "./curricula/datasets/curriculum_10M_2024_eval"
args["epochs_per_stage"] = 10

curriculum = {
    "C1": ["childes.train"],
    "C2": ["open_subtitles.train", "bnc_spoken.train"],
    "C3": ["switchboard.train"],
    "C4": ["gutenberg.train"],
    "C5": [ "simple_wiki.train"]
} # TODO

In [11]:
datasets_stages = []
orders_stages = []

torch.manual_seed(0)
for stage, files in curriculum.items():
    d = datasets.concatenate_datasets(
            [
                load_dataset("text", data_files =os.path.join(args["raw_dataset_folder_babylm"], file))["train"] 
                for file in files
            ]
        )
    d = d.shuffle() # we shuffle with the stage
    datasets_stages.append(d)
    offset = orders_stages[-1][-1]+1 if len(orders_stages) else 0
    for i in range(0,args["epochs_per_stage"]):
        indices = torch.arange(offset,len(d)+offset) # and then shuffle again (across epochs within stages)
        orders_stages.append(indices[torch.randperm(len(indices))])
    


dataset = datasets.concatenate_datasets(datasets_stages)
dataset.save_to_disk(args["dataset_folder"])
# assert torch.equal(torch.cat([o.flatten() for o in orders_stages]),torch.arange(0,len(dataset)))

torch.save(orders_stages, args["curriculum_path"])

len_full_dataset = len(dataset)
args["epoch_equivalents"] = sum([len(o) for o in orders_stages]) / len_full_dataset
args["epochs"] = len(orders_stages)

dataset = dataset.shuffle()
dataset = datasets.Dataset.from_dict(dataset[0:len(dataset)//100]) # TODO
dataset.save_to_disk(args["eval_dataset_folder"])


with open("./configs/curriculum_10M_2024", 'w') as f:
    json.dump(args, f)
args

Saving the dataset (0/1 shards):   0%|          | 0/1179014 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/11790 [00:00<?, ? examples/s]

{'dataset_folder': './curricula/datasets/curriculum_10M_2024',
 'raw_dataset_folder_babylm': './train_10M',
 'curriculum_path': './curricula/curriculum_10M_2024',
 'eval_dataset_folder': './curricula/datasets/curriculum_10M_2024_eval',
 'epochs_per_stage': 10,
 'epoch_equivalents': 10.0,
 'epochs': 50}

|     Groups     |Version|Filter|n-shot|Metric|Value |   |Stderr|
|----------------|-------|------|-----:|------|-----:|---|-----:|
|blimp_supplement|N/A    |none  |     0|acc   |0.4584|±  |0.0069|
|blimp_filtered  |N/A    |none  |     0|acc   |0.5273|±  |0.0020|

### Random Shuffle

This mimicks the default behaviour of Huggingface's `Trainer`: Randomly shuffle dataset after each epoch.
All data is shown at each epoch.

In [12]:
args = {
    "dataset_folder": "./curricula/datasets/curriculum_10M_2024",# reuse one created above
   
}

In [13]:
args["raw_dataset_folder_babylm"] = "./train_10M"
args["curriculum_path"] = "./curricula/random_10M_2024"
args["eval_dataset_folder"] = "./curricula/datasets/curriculum_10M_2024_eval"
args["epochs"] = 10

In [14]:
from datasets import load_dataset
import datasets
dataset = datasets.load_from_disk(args["dataset_folder"]) # reuse one created above

In [15]:
assert torch.equal(torch.sort(torch.randperm(len(dataset))).values, torch.arange(0,len(dataset)))

In [16]:
args["epochs"] = 10
import torch 
torch.manual_seed(0)
random_order = torch.stack([torch.randperm(len(dataset)) for _ in range(0,args["epochs"])])
torch.save(random_order, "./curricula/random_10M_2024")
with open("./configs/random_10M_2024", 'w') as f:
    json.dump(args, f)
args

{'dataset_folder': './curricula/datasets/curriculum_10M_2024',
 'raw_dataset_folder_babylm': './train_10M',
 'curriculum_path': './curricula/random_10M_2024',
 'eval_dataset_folder': './curricula/datasets/curriculum_10M_2024_eval',
 'epochs': 10}

|     Groups     |Version|Filter|n-shot|Metric|Value |   |Stderr|
|----------------|-------|------|-----:|------|-----:|---|-----:|
|blimp_supplement|N/A    |none  |     0|acc   |0.4401|±  |0.0069|
|blimp_filtered  |N/A    |none  |     0|acc   |0.4579|±  |0.0019|


### Curriculum Convolution with lognorm 

## Test random

In [17]:
args = {
    "dataset_folder": "./curricula/datasets/test",   
}
args["raw_dataset_folder_babylm"] = "./train_10M"
args["curriculum_path"] = "./curricula/test_random_curriculum"
args["eval_dataset_folder"] = "./curricula/datasets/test_eval"
args["epochs"] = 3

In [18]:
import datasets
import torch 
torch.manual_seed(0)
import json
#dataset = load_dataset("text", data_dir ="/data/loriss21dm/babylm/train_test", cache_dir='/data/loriss21dm/cache')["train"] 
dataset = datasets.load_from_disk("./curricula/datasets/curriculum_10M_2024") # reuse one created above
dataset = dataset.shuffle()
dataset = datasets.Dataset.from_dict(dataset[0:len(dataset)//1000]) # TODO
dataset.save_to_disk(args["dataset_folder"])

random_order = torch.stack([torch.randperm(len(dataset)) for _ in range(0,args["epochs"])])
torch.save(random_order, args["curriculum_path"])


dataset = datasets.Dataset.from_dict(dataset[0:len(dataset)//10]) # TODO
dataset.save_to_disk(args["eval_dataset_folder"])

with open("./configs/test", 'w') as f:
    json.dump(args, f)
args

Saving the dataset (0/1 shards):   0%|          | 0/1179 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/117 [00:00<?, ? examples/s]

{'dataset_folder': './curricula/datasets/test',
 'raw_dataset_folder_babylm': './train_10M',
 'curriculum_path': './curricula/test_random_curriculum',
 'eval_dataset_folder': './curricula/datasets/test_eval',
 'epochs': 3}